In [1]:
from google.colab import drive

import shutil

drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
!pip install transformers
!pip install gradio
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments, AutoTokenizer, AutoModelWithLMHead


import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random

from tqdm.auto import tqdm

from nltk.translate.bleu_score import corpus_bleu, sentence_bleu

import re

In [5]:
tokenizer = AutoTokenizer.from_pretrained("t5-base")
checkpoint = torch.load('drive/MyDrive/best_model.pt', map_location=torch.device('cpu'))
model=AutoModelWithLMHead.from_pretrained("t5-base", pad_token_id=tokenizer.eos_token_id)
model.load_state_dict(checkpoint)

/usr/local/lib/python3.8/dist-packages/transformers/models/auto/modeling_auto.py:1177: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

<All keys matched successfully>

In [14]:

import gradio as gr
from nltk import tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [19]:


def greet(sentence):
    data_sentence_list=tokenize.sent_tokenize(sentence)

    res=""
    for sentence in data_sentence_list:
      input = tokenizer.encode(sentence)
      output = model.generate(input_ids=torch.tensor([input]).to("cpu"), max_new_tokens=40, do_sample=True)
      output = output[0]
      output = output[1:-1]
      out_str=tokenizer.decode(output)
      res=res+out_str

    return  res

demo = gr.Interface(fn=greet, inputs="text", outputs="label")

demo.launch(debug=True)   

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.
